# NPL load text demo

This notebook is based on Tensorflow tutorial's load text: https://www.tensorflow.org/tutorials/load_data/text 

In [1]:
# import library 
import collections
import pathlib
import re
import string

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds
# import tensorflow_text as tf_text

In [2]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz

--2021-08-17 09:27:17--  https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.193.128, 173.194.195.128, 64.233.191.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6053168 (5.8M) [application/x-gzip]
Saving to: ‘stack_overflow_16k.tar.gz’

stack_overflow_16k. 100%[===================>]   5.77M  --.-KB/s    in 0.09s   

2021-08-17 09:27:17 (63.4 MB/s) - ‘stack_overflow_16k.tar.gz’ saved [6053168/6053168]



In [3]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

dataset_dir = utils.get_file(
    fname="stack_overflow",
    origin=data_url,
    untar=True, # extract the file
    cache_dir='',
    cache_subdir='')

dataset_dir = pathlib.Path(dataset_dir).parent

6053888/6053168 [==============================] - 0s 0us/step


In [4]:
!pwd

/content


In [5]:
list(dataset_dir.iterdir())

[PosixPath('/tmp/.keras/README.md'),
 PosixPath('/tmp/.keras/stack_overflow.tar.gz'),
 PosixPath('/tmp/.keras/test'),
 PosixPath('/tmp/.keras/train')]

In [6]:
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

[PosixPath('/tmp/.keras/train/java'),
 PosixPath('/tmp/.keras/train/csharp'),
 PosixPath('/tmp/.keras/train/javascript'),
 PosixPath('/tmp/.keras/train/python')]

In [7]:
sample_file = train_dir/'python/1755.txt'
with open(sample_file) as f:
  print(f.read())

why does this blank program print true x=true.def stupid():.    x=false.stupid().print x



In [8]:
train_python_dir = dataset_dir/'train'/'python'
train_python = list(train_python_dir.iterdir())
len(train_python)

2000

In [9]:
batch_size = 32
seed = 42

raw_train_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [10]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(5):
    print(f"Question: {text_batch.numpy()[i]}")
    print(f"Label: {label_batch.numpy()[i]}")

Question: b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default cons

In [11]:
class_names = raw_train_ds.class_names
class_names

['csharp', 'java', 'javascript', 'python']

In [12]:
for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


In [13]:
# test_dir = dataset_dir/"test"
train_dir = dataset_dir/"train"
batch_size = 32
seed = 42
raw_val_ds = preprocessing.text_dataset_from_directory(train_dir,
                                                       batch_size=batch_size,
                                                       validation_split=0.2,
                                                       seed=42,
                                                       subset="validation")

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [14]:
test_dir = dataset_dir/"test"

raw_test_ds = preprocessing.text_dataset_from_directory(test_dir,
                                                        batch_size=batch_size)

Found 8000 files belonging to 4 classes.


## Preprocess the data

The default vectorization mode is `int`. This outputs integer indices (one per token) aka **Tokenization**. This mode can be used to build models that take word order into account. You can also use other modes, like `binary`, to build bag-of-word models.

In [15]:
# Binary mode
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

In [16]:
# Int mode 
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_SEQUENCE_LENGTH
)

In [17]:
# Make a text-only dataset (without labels) then call adapt
train_text = raw_train_ds.map(lambda text, label: text)

binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

In [18]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [19]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

In [20]:
# Retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)

Question tf.Tensor(b'"what is the difference between these two ways to create an element? var a = document.createelement(\'div\');..a.id = ""mydiv"";...and..var a = document.createelement(\'div\').id = ""mydiv"";...what is the difference between them such that the first one works and the second one doesn\'t?"\n', shape=(), dtype=string)
Label tf.Tensor(2, shape=(), dtype=int32)


In [21]:
print("'binary' vectorized question:", 
      binary_vectorize_text(first_question, first_label)[0])

'binary' vectorized question: tf.Tensor([[1. 1. 0. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [22]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])

'int' vectorized question: tf.Tensor(
[[ 55   6   2 410 211 229 121 895   4 124  32 245  43   5   1   1   5   1
    1   6   2 410 211 191 318  14   2  98  71 188   8   2 199  71 178   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0


In [23]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [24]:
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

### Train the binary model (Bag of words)

In [25]:
nums_classes = len(class_names)

In [48]:
# create the model
binary_model = tf.keras.Sequential([
  layers.Input(shape=(1, VOCAB_SIZE)),
  # layers.Embedding(VOCAB_SIZE, output_dim=64, mask_zero=True),
  # layers.GlobalMaxPool1D(),
  layers.Dense(nums_classes)
], name="binary_model_extra")
# compile the model
binary_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [49]:
binary_model.summary()

Model: "binary_model_extra"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1, 4)              40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________


In [50]:
# fit the model
binary_history = binary_model.fit(binary_train_ds,
                              epochs=10,
                              validation_data=binary_val_ds)

Epoch 1/10
200/200 [==============================] - 1s 4ms/step - loss: 1.1196 - accuracy: 0.6456 - val_loss: 0.9137 - val_accuracy: 0.7794
Epoch 2/10
200/200 [==============================] - 1s 3ms/step - loss: 0.7792 - accuracy: 0.8202 - val_loss: 0.7497 - val_accuracy: 0.7994
Epoch 3/10
200/200 [==============================] - 1s 3ms/step - loss: 0.6281 - accuracy: 0.8602 - val_loss: 0.6641 - val_accuracy: 0.8150
Epoch 4/10
200/200 [==============================] - 1s 3ms/step - loss: 0.5348 - accuracy: 0.8863 - val_loss: 0.6107 - val_accuracy: 0.8263
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4688 - accuracy: 0.9045 - val_loss: 0.5740 - val_accuracy: 0.8375
Epoch 6/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4185 - accuracy: 0.9164 - val_loss: 0.5473 - val_accuracy: 0.8406
Epoch 7/10
200/200 [==============================] - 1s 3ms/step - loss: 0.3782 - accuracy: 0.9286 - val_loss: 0.5272 - val_accuracy: 0.8425
Epoch 

In [27]:
# fit the model
binary_history = binary_model.fit(binary_train_ds,
                              epochs=10,
                              validation_data=binary_val_ds)

Epoch 1/10
200/200 [==============================] - 4s 16ms/step - loss: 1.1185 - accuracy: 0.6388 - val_loss: 0.9158 - val_accuracy: 0.7806
Epoch 2/10
200/200 [==============================] - 1s 3ms/step - loss: 0.7786 - accuracy: 0.8194 - val_loss: 0.7516 - val_accuracy: 0.8037
Epoch 3/10
200/200 [==============================] - 1s 3ms/step - loss: 0.6277 - accuracy: 0.8611 - val_loss: 0.6658 - val_accuracy: 0.8144
Epoch 4/10
200/200 [==============================] - 1s 3ms/step - loss: 0.5344 - accuracy: 0.8870 - val_loss: 0.6123 - val_accuracy: 0.8269
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4684 - accuracy: 0.9038 - val_loss: 0.5755 - val_accuracy: 0.8325
Epoch 6/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4181 - accuracy: 0.9156 - val_loss: 0.5488 - val_accuracy: 0.8350
Epoch 7/10
200/200 [==============================] - 1s 3ms/step - loss: 0.3779 - accuracy: 0.9284 - val_loss: 0.5286 - val_accuracy: 0.8375
Epoch

In [28]:
# evaluate the model
binary_results = binary_model.evaluate(binary_test_ds)
binary_results

250/250 [==============================] - 3s 11ms/step - loss: 0.5178 - accuracy: 0.8150


[0.5178357362747192, 0.8149999976158142]

### Train the int model (tokenization)

In [29]:
nums_classes = len(class_names)

In [30]:
# Create the model
int_model = tf.keras.Sequential([
  layers.Embedding(VOCAB_SIZE, output_dim=64, mask_zero=True),
  layers.Conv1D(filters=64,
                kernel_size=5,
                padding="valid",
                activation="relu",
                strides=1),
  layers.GlobalMaxPool1D(),
  layers.Dense(nums_classes)
])

# Compile the model
int_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)


In [31]:
int_history = int_model.fit(int_train_ds,
                            validation_data=int_val_ds,
                            epochs=10)

Epoch 1/10
200/200 [==============================] - 11s 52ms/step - loss: 1.1108 - accuracy: 0.5186 - val_loss: 0.7290 - val_accuracy: 0.7081
Epoch 2/10
200/200 [==============================] - 8s 41ms/step - loss: 0.6068 - accuracy: 0.7652 - val_loss: 0.5230 - val_accuracy: 0.8100
Epoch 3/10
200/200 [==============================] - 8s 41ms/step - loss: 0.3703 - accuracy: 0.8786 - val_loss: 0.4592 - val_accuracy: 0.8288
Epoch 4/10
200/200 [==============================] - 8s 41ms/step - loss: 0.2128 - accuracy: 0.9478 - val_loss: 0.4548 - val_accuracy: 0.8313
Epoch 5/10
200/200 [==============================] - 8s 41ms/step - loss: 0.1122 - accuracy: 0.9795 - val_loss: 0.4753 - val_accuracy: 0.8281
Epoch 6/10
200/200 [==============================] - 8s 41ms/step - loss: 0.0549 - accuracy: 0.9939 - val_loss: 0.5049 - val_accuracy: 0.8250
Epoch 7/10
200/200 [==============================] - 8s 42ms/step - loss: 0.0267 - accuracy: 0.9986 - val_loss: 0.5381 - val_accuracy: 0.823